In [1]:
import pandas as pd
import tweepy
import os
import glob
import time
import random
import numpy as np

In [ ]:
# Hello, This is Xiaoyu Wei, and I want to share what I was doing with these codes.^^ 

# In the beginning, we directly chose the dataset on the IEEE official website, which is the daily Twitter dataset since the outbreak of the Covid-19, 
# equivalent to more than 900 days and more than 1 million tweets per day. This IEEE dataset has only Twitter token number and no Twitter text body information. 
# We need to use algorithm to turn the Twitter number into text and then perform sentiment analysis.

# Patrick was excellent and wrote the code on how to make the Twitter token number into regular text, 
# But the algorithm to translate one numbers file into Tweets text took 9 hours! (remember, we need to do with 900+ files! 900*9h ... ) 
# (His work is not trivial, he is smart. 9 hours' algorithm has already been optimized ! )

# So, I rewrote the codes and spent 3 days and 3 nights constantly optimizing them (I wrote the codes during the day and tested them at night to see
# if they would report errors, I needed to wake up several times in the middle of the night to see if report errors....) 
# and finally, 
# this version of the algorithm downside can translate 1 file (with over 1 million tweets)  in less than 1 hour ! Big progress!

# until now, there's no people on the internet, GitHub, have achieved a similar job before.
# If other research teams in the future need to analyze the IEEE Covid-19 Twitter data, this optimized method code can significantly improve their work efficiency.;)

# .....But Since we used other already well-done datasets that Patrick found later, these efforts could have been more decisive in our paper. 
# But I learned a lot from these optimization attempts!! ;D 

In [2]:
consumer_key = "aRmOhzcdCDiWdVts9Q0K45u5K"
consumer_secret = "TFsYHY5MBfMfzlXy33vsuttCjIzdKtKQB6Nalq3X2NFrfiw8m3"
access_token = "1573003732987613187-ODn0j3HDW0BaYNJZll0DEvhODhGAZs"
access_token_secret = "DFeLQprKhBQWj6VWth7DnMxoBYUwZhYiC4Jd3LBoSpcyw"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
keywords = ["vaccine", "vax", "vacc", "sinopharm", "sputnik", "pfizer", "moderna", "astrazeneca",
            "vaxzevria", "spikevax", "novavax", "nuvaxovid", "biontech", "comirnaty",
            "bbil", "covaxin", "janssen", "johnson", "covishield", "BBIBP-CorV",
            "sinovac", "coronavac", "gamaleya"]

In [3]:
def generate_text_files(dataframe, batch_size):
    last_tweet_num = 0
    counter = 0
    length = len(dataframe)
    batch_number = int(length/batch_size)
    num_left_over = length-batch_number*batch_size
    for i in range(0, len(dataframe), batch_size):
        cur_batch = dataframe[i:i+batch_size]
        try:          
            statuses = api.lookup_statuses(cur_batch, tweet_mode='extended')
        except:
            print("---Need sometime to make next fetch---")
            time.sleep(600)
            print("---Restart Fetching---")
            statuses = api.lookup_statuses(cur_batch, tweet_mode='extended')
            
        for status in statuses:
            if(status.full_text is not None):
                tweet_text = status.full_text
                tweet_time = status.created_at
                tweet_coords = status.coordinates
                tweet_place = status.place
                tweet_id = status.id

                contains_keyword = False

                for keyword in keywords:
                    if(keyword.lower() in tweet_text.lower()):
                        contains_keyword = True
                
                if(contains_keyword):
                    outfile = open("./new_tweet_" + str(last_tweet_num) + ".txt", "w")
                    outfile.write(str(tweet_text))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_time))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_coords))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_place))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_id))

                    outfile.close()
                    last_tweet_num += 1

        counter = counter+1
        
    for i in reversed(range(len(dataframe)-num_left_over,len(dataframe))):
        cur_batch = dataframe[batch_size*batch_number:len(dataframe)]
        try:          
            statuses = api.lookup_statuses(cur_batch, tweet_mode='extended')
        except:
            print("---Need sometime to make next fetch---")
            time.sleep(600)
            print("---Restart Fetching---")
            statuses = api.lookup_statuses(cur_batch, tweet_mode='extended')
            
        for status in statuses:
            if(status.full_text is not None):
                tweet_text = status.full_text
                tweet_time = status.created_at
                tweet_coords = status.coordinates
                tweet_place = status.place
                tweet_id = status.id

                contains_keyword = False

                for keyword in keywords:
                    if(keyword.lower() in tweet_text.lower()):
                        contains_keyword = True

                if(contains_keyword):
                    name = "new_tweet_" + str(last_tweet_num) + ".txt"
                    outfile = open("./tweets/new_tweets/new_tweet_" + str(last_tweet_num) + ".txt", "w")

                    outfile.write(str(tweet_text))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_time))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_coords))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_place))
                    outfile.write("\n\n")
                    outfile.write(str(tweet_id))

                    outfile.close()
                    last_tweet_num += 1
        counter = counter+1

In [4]:
def get_id_and_tweets(path):
    extension = 'csv'
    files_names = glob.glob('*.{}'.format(extension))
    current_path = os.getcwd()
    print(current_path)
    for i in range(len(files_names)):

        cur_dataframe = pd.read_csv(files_names[i])
        print("---Creating folder for",files_names[0][:-4], "---")
        if not os.path.exists(files_names[0][:-4]):
            os.mkdir(files_names[0][:-4])
        else:
            print("---Folder,", files_names[0][:-4], "Already Exists!---")
       
        print("---Changing Directory---")
        os.chdir(files_names[i][:-4])
        
        
        
        sample_ids = list(cur_dataframe.iloc[:,0])
        print("---Fetching Tweets...---")
        generate_text_files(sample_ids, batch_size=100)
        print("---Compeleting Fetching Tweets for, ", files_names[i], "---")
        print("---Returning to parent directory---")
        
        os.chdir('C:\\Users\\xywei\\Desktop\\Xiaoyu DataScience\\tweets\\')
    
    print("---Done Fetching---")

In [ ]:
os.chdir('C:\\Users\\xywei\\Desktop\\Xiaoyu DataScience\\tweets\\')
get_id_and_tweets('./tweets/')